# Average feature vectors and save the output on disk

In [1]:
import glob
import os
import numpy as np
import tqdm

In [2]:
MAP_SIZE = 1099511627776
features_dir = "datasets/ME/features_100/dc/ME_trainval_resnext152_faster_rcnn_genome.lmdb"

In [3]:
infiles = glob.glob(os.path.join(features_dir, "*"))

In [4]:
# check and remove files with errors

count_ = {}
trun = []
for infile in tqdm.tqdm(infiles):
    try:
        reader = np.load(infile, allow_pickle=True)
        fid = reader.item().get("image_id")
        id_ = int(fid.split('_')[0])
        s = int(fid.split('_')[1])
        if id_ in count_:
            count_[id_] += 1
        else:
            count_[id_] = 1
    except:
        trun.append(os.path.splitext(os.path.basename(infile))[0])
        os.remove(infile)

100%|██████████| 40000/40000 [03:08<00:00, 212.08it/s]


In [5]:
len(trun)

0

In [6]:
len(count_)

8000

In [37]:
feats = {}

for infile in tqdm.tqdm(infiles):
    reader = np.load(infile, allow_pickle=True)
    fid = reader.item().get("image_id")
    id_ = int(fid.split('_')[0])
    s = int(fid.split('_')[1])
    if id_ in feats:
        feats[id_]["image_id"] = str(id_)
        # perform an average of the existing feature and the new one
        feats[id_]["features"] = np.mean( np.array([ feats[id_]["features"], reader.item().get("features") ]), axis=0 )
    else:
        # initialize the sample
        feats[id_] = reader.item()

100%|██████████| 40000/40000 [04:51<00:00, 137.38it/s] 


In [38]:
len(feats)

8000

In [50]:
features_dir = "datasets/ME/features_100/dc/ME_trainval_resnext152_faster_rcnn_genome.lmdb/avg"

for img_id, feat in tqdm.tqdm(feats.items()):
    file_name = str(img_id) + ".npy"
    np.save(os.path.join(features_dir, file_name), feat)

100%|██████████| 8000/8000 [00:15<00:00, 532.59it/s] 
